# ENN Sequence Model Test

In [1]:
import os, sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import mnist
from IPython.display import Image
from time import perf_counter as perf_time
from keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow_engram.layers import Engram
from tensorflow_engram.models import engram_classifier
from tensorflow_engram.utils import HebbianTraceMonitor, plot_hebbian_trace

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

np.random.seed(42)
tf.random.set_seed(42)

2025-05-22 13:25:55.461575: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 13:25:55.697306: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 13:25:55.789623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747920355.929034    7244 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747920355.975588    7244 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747920356.268583    7244 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
%pip install datasets
from datasets import load_dataset

  Using cached aiohttp-3.11.18-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.6.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached propcache-0.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached yarl-1.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (72 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 62.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 32.3 MB/s eta 0:00:00
Using cached aiohttp-3.11.18-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cac

In [ ]:
dataset = load_dataset("wikitext", "wikitext-103-v1", split="train")

# Preprocess the dataset
def preprocess_text(text):
    # Tokenize the text
    tokens = text.split()
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    return tokens

# Apply preprocessing
dataset = dataset.map(lambda x: {'tokens': preprocess_text(x['text'])})

# Create a vocabulary
vocab = set()
for example in dataset:
    vocab.update(example['tokens'])
vocab = sorted(vocab)
vocab_size = len(vocab)
word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in enumerate(vocab)}

# Convert tokens to indices
def tokens_to_indices(tokens):
    return [word_to_index[token] for token in tokens if token in word_to_index]
dataset = dataset.map(lambda x: {'indices': tokens_to_indices(x['tokens'])})

# Create sequences
def create_sequences(indices, seq_length=32):
    sequences = []
    for i in range(0, len(indices) - seq_length, seq_length):
        sequences.append(indices[i:i + seq_length])
    return sequences
dataset = dataset.map(lambda x: {'sequences': create_sequences(x['indices'])})

# Flatten the dataset
dataset = dataset.map(lambda x: {'sequences': [item for sublist in x['sequences'] for item in sublist]})

Map: 100%|██████████| 1801350/1801350 [03:41<00:00, 8148.74 examples/s] 


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1441080,) + inhomogeneous part.

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Split the dataset into training and testing sets
X_train, X_test = train_test_split(dataset['sequences'], test_size=0.2, random_state=42)

# Pad sequences to ensure uniform length
max_sequence_length = 32  # Define the maximum sequence length
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

# Convert to numpy arrays
X_train = np.array(X_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)

# Reshape the data to fit the model
X_train = X_train.reshape(-1, max_sequence_length, 1)  # 32 time steps, each with 1 feature
X_test = X_test.reshape(-1, max_sequence_length, 1)

# Apply scaler for additional normalization
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

In [ ]:
# Shift the target sequences by one time step
y_train = np.roll(X_train, shift=-1, axis=1)
y_test = np.roll(X_test, shift=-1, axis=1)

# Flatten the target sequences
y_train = y_train.reshape(-1)  # Flatten to (batch_size * sequence_length,)
y_test = y_test.reshape(-1)

# Create the model
model = engram_classifier(
    input_shape=(32, 1),  # 32 time steps, each with 1 feature
    hidden_dim=128,
    memory_size=64,  # Increased memory size
    num_classes=vocab_size,
    return_states=True,
    hebbian_lr=0.05  # Higher learning rate for Hebbian updates
)

trace_callback = HebbianTraceMonitor(
    X_train[:32], 
    log_dir="examples/out/wikitext_hebbian_trace", 
    verbose=1, 
    plot_every=100, 
    skip_first_plot=True
)
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-5
)

# Compile and train the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True, expand_nested=True, to_file='examples/out/wikitext_model.png')
Image(filename='examples/out/wikitext_model.png', width=800)

# Train the model
start_time = perf_time()
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[trace_callback, early_stopping, lr_scheduler]
)
end_time = perf_time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}") 
print(f"Test Accuracy: {accuracy:.4f}")

# Plot the training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig('examples/out/wikitext_training_history.png')
plt.show()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ engram_5 (Engram)               │ (None, 128)            │       214,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 229463)         │    29,600,727 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,814,999 (113.74 MB)

 Trainable params: 29,806,807 (113.70 MB)

 Non-trainable params: 8,192 (32.00 KB)

HebbianTraceMonitor initialized.
Epoch 1/10
   81/36027 ━━━━━━━━━━━━━━━━━━━━ 2:25:49 243ms/step - accuracy: 0.5539 - loss: 9.0562

KeyboardInterrupt: 